### 1. Load Packages

In [1]:
import openai
import pandas as pd
import re
import json

### 2. Load Tokens

In [2]:
openai.api_key = "To be filled"

### 3. Load Data or Text

In [3]:
processed_df1 = pd.read_csv('Demonstrators_df1.csv')
processed_df2 = pd.read_csv('Demonstrators_df2.csv')

In [4]:
processed_df1

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons
0,60900,we already have congestion pricing for taxis &...,car,positive,congestion pricing is intended to encourage th...
1,60901,an the best cannot be that much i am sorry htt...,na,NaN,NaN
2,60902,handicap uber's is getting out of hand,uber,negative,increase in number of handicap ubers causing i...
3,60903,bitchh i would calmly get in a cab laugh ill b...,taxi,positive,excited about getting into a cab
4,60904,take a cab. you are getting hepatitis walking ...,taxi,negative,"unsafe environment for walking, implying the n..."
...,...,...,...,...,...
995,61895,so...i confess ...i walked to target &amp; bou...,uber,positive,"excited about buying a bigger tv for the team,..."
996,61896,"uber, ferry, plane, train today",na,NaN,NaN
997,61897,"hi, i order some food about a hour ago and i h...",na,NaN,NaN
998,61898,i was there. it was an uber. none of the other...,uber,neutral,na


In [5]:
processed_df2

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons
0,177000,please kick japan out of my country they are s...,na,NaN,NaN
1,177001,once i learned about correctional exercises an...,na,NaN,NaN
2,177002,empty train cars &gt;&gt;,subway,positive,empty train cars
3,177003,that emerald parking lot is about 1 mile walk ...,na,NaN,NaN
4,177004,"that is going to take time, i need car by end ...",car,neutral,"expresses need for a car by end of the year, i..."
...,...,...,...,...,...
995,177995,how about getting all trash off the sidewalk? ...,car,negative,presence of trash on sidewalk and street causi...
996,177996,get trash off the sidewalk. trash corals in th...,car,negative,"congestion caused by cars, lack of space on si..."
997,177997,people of philadelphia find trae young's hotel...,car,negative,disruptive behavior of honking car horns all n...
998,177998,ring rain coming back for more and do not you ...,car,positive,expresses joy and appreciation for the time sp...


### 4. Self-Verfication

In [6]:
def self_verify(tweet, travel_mode, sentiment, reasons):
    # Building the structured prompt with included examples as demonstrators
    verification_prompt = f"""
    System: You are a highly intelligent assistant trained to verify the accuracy of travel mode analysis. Provide your verification based on the analysis and instructions below:
    
    Instructions: Assess the analysis for correctness based on relevance and accuracy of the travel mode, sentiment, and reasons given. Respond with 'correct', 'incorrect', or 'undetermined'. All responses must be in lowercase.

    Examples:
    - Example 1: Tweet: "the best eats and doordash drivers abuse tf out this button https: and and t.co and xefkdiaef6"
      Analysis: Travel Mode: "car", Sentiment: "negative", Reasons: "drivers abuse the button".
      Verification: "correct"

    - Example 2: Tweet: "i do not give a fuck about my the best rating the best drivers make it so hard to sit still"
      Analysis: Travel Mode: "na", Sentiment: "", Reasons: "".
      Verification: "incorrect"
      
    Given Analysis:
    - Tweet: {tweet}
    - Travel Mode: {travel_mode}
    - Sentiment: {sentiment}
    - Reasons: {reasons}
    
    Is this analysis 'correct' or 'incorrect' or 'undetermined'?
    """

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{
            "role": "system",
            "content": "You're a highly intelligent assistant. Please read the following analysis carefully and provide your verification."
        }, {
            "role": "user",
            "content": verification_prompt
        }],
        temperature=0,  # Use 0 for deterministic output
        max_tokens=50
    )

    # Simplifying the response check
    text_response = response.choices[0].message.content.strip().lower()
    if "correct" in text_response:
        return "correct"
    elif "incorrect" in text_response:
        return "incorrect"
    else:
        return "undetermined"  # Use this if the response isn't clear

In [7]:
#test
self_verify(processed_df1.iloc[0]['Tweet'],processed_df1.iloc[0]['Travel Mode'], processed_df1.iloc[0]['Sentiment'], processed_df1.iloc[0]['Reasons'])

'correct'

**df1**

In [8]:
#df1
processed_df1['Self Verification Result'] = processed_df1.apply(
    lambda row: self_verify(row['Tweet'], row['Travel Mode'], row['Sentiment'], row['Reasons']), axis=1)

In [9]:
processed_df1

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons,Self Verification Result
0,60900,we already have congestion pricing for taxis &...,car,positive,congestion pricing is intended to encourage th...,correct
1,60901,an the best cannot be that much i am sorry htt...,na,NaN,NaN,correct
2,60902,handicap uber's is getting out of hand,uber,negative,increase in number of handicap ubers causing i...,correct
3,60903,bitchh i would calmly get in a cab laugh ill b...,taxi,positive,excited about getting into a cab,correct
4,60904,take a cab. you are getting hepatitis walking ...,taxi,negative,"unsafe environment for walking, implying the n...",correct
...,...,...,...,...,...,...
995,61895,so...i confess ...i walked to target &amp; bou...,uber,positive,"excited about buying a bigger tv for the team,...",correct
996,61896,"uber, ferry, plane, train today",na,NaN,NaN,correct
997,61897,"hi, i order some food about a hour ago and i h...",na,NaN,NaN,correct
998,61898,i was there. it was an uber. none of the other...,uber,neutral,na,correct


In [10]:
processed_df1.to_csv('Week 11 result_60900_61899((after self-verification).csv', index=False)

In [11]:
processed_df1['Self Verification Result'].value_counts().get('correct', 0)

1000

**df2**

In [12]:
#df2
processed_df2['Self Verification Result'] = processed_df2.apply(
    lambda row: self_verify(row['Tweet'], row['Travel Mode'], row['Sentiment'], row['Reasons']), axis=1)

In [13]:
processed_df2

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons,Self Verification Result
0,177000,please kick japan out of my country they are s...,na,NaN,NaN,correct
1,177001,once i learned about correctional exercises an...,na,NaN,NaN,correct
2,177002,empty train cars &gt;&gt;,subway,positive,empty train cars,correct
3,177003,that emerald parking lot is about 1 mile walk ...,na,NaN,NaN,correct
4,177004,"that is going to take time, i need car by end ...",car,neutral,"expresses need for a car by end of the year, i...",correct
...,...,...,...,...,...,...
995,177995,how about getting all trash off the sidewalk? ...,car,negative,presence of trash on sidewalk and street causi...,correct
996,177996,get trash off the sidewalk. trash corals in th...,car,negative,"congestion caused by cars, lack of space on si...",correct
997,177997,people of philadelphia find trae young's hotel...,car,negative,disruptive behavior of honking car horns all n...,correct
998,177998,ring rain coming back for more and do not you ...,car,positive,expresses joy and appreciation for the time sp...,correct


In [14]:
processed_df2.to_csv('Week 11 result_177000_177999(after self-verification).csv', index=False)

In [15]:
processed_df2['Self Verification Result'].value_counts().get('correct', 0)

999

### 5. Cross-Verfication

In [16]:
def cross_verify(tweet, travel_mode, sentiment, reasons):
    # Building the structured prompt with included examples as demonstrators
    verification_prompt = f"""
    System: You are a highly intelligent assistant trained to verify the accuracy of travel mode analysis. Provide your verification based on the analysis and instructions below:
    
    Instructions: Assess the analysis for correctness based on relevance and accuracy of the travel mode, sentiment, and reasons given. Respond with 'correct', 'incorrect', or 'undetermined'. All responses must be in lowercase.

    Examples:
    - Example 1: Tweet: "the best eats and doordash drivers abuse tf out this button https: and and t.co and xefkdiaef6"
      Analysis: Travel Mode: "car", Sentiment: "negative", Reasons: "drivers abuse the button".
      Verification: "correct"

    - Example 2: Tweet: "i do not give a fuck about my the best rating the best drivers make it so hard to sit still"
      Analysis: Travel Mode: "na", Sentiment: "", Reasons: "".
      Verification: "incorrect"
      
    Given Analysis:
    - Tweet: {tweet}
    - Travel Mode: {travel_mode}
    - Sentiment: {sentiment}
    - Reasons: {reasons}
    
    Is this analysis 'correct' or 'incorrect' or 'undetermined'?
    """

    response = openai.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{
            "role": "system",
            "content": "You're a highly intelligent assistant. Please read the following analysis carefully and provide your verification."
        }, {
            "role": "user",
            "content": verification_prompt
        }],
        temperature=0,  # Use 0 for deterministic output
        max_tokens=50
    )

    # Simplifying the response check
    text_response = response.choices[0].message.content.strip().lower()
    if "correct" in text_response:
        return "correct"
    elif "incorrect" in text_response:
        return "incorrect"
    else:
        return "undetermined"  # Use this if the response isn't clear

In [17]:
#test
cross_verify(processed_df1.iloc[0]['Tweet'],processed_df1.iloc[0]['Travel Mode'], processed_df1.iloc[0]['Sentiment'], processed_df1.iloc[0]['Reasons'])

'correct'

**df1**

In [18]:
#df1
processed_df1['Cross Verification Result'] = processed_df1.apply(
    lambda row: cross_verify(row['Tweet'], row['Travel Mode'], row['Sentiment'], row['Reasons']), axis=1)

In [19]:
processed_df1

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons,Self Verification Result,Cross Verification Result
0,60900,we already have congestion pricing for taxis &...,car,positive,congestion pricing is intended to encourage th...,correct,correct
1,60901,an the best cannot be that much i am sorry htt...,na,NaN,NaN,correct,correct
2,60902,handicap uber's is getting out of hand,uber,negative,increase in number of handicap ubers causing i...,correct,correct
3,60903,bitchh i would calmly get in a cab laugh ill b...,taxi,positive,excited about getting into a cab,correct,correct
4,60904,take a cab. you are getting hepatitis walking ...,taxi,negative,"unsafe environment for walking, implying the n...",correct,correct
...,...,...,...,...,...,...,...
995,61895,so...i confess ...i walked to target &amp; bou...,uber,positive,"excited about buying a bigger tv for the team,...",correct,correct
996,61896,"uber, ferry, plane, train today",na,NaN,NaN,correct,correct
997,61897,"hi, i order some food about a hour ago and i h...",na,NaN,NaN,correct,correct
998,61898,i was there. it was an uber. none of the other...,uber,neutral,na,correct,correct


In [20]:
processed_df1.to_csv('Week 11 result_60900_61899((after cross-verification).csv', index=False)

In [21]:
processed_df1['Cross Verification Result'].value_counts().get('correct', 0)

1000

**df2**

In [22]:
#df2
processed_df2['Cross Verification Result'] = processed_df2.apply(
    lambda row: cross_verify(row['Tweet'], row['Travel Mode'], row['Sentiment'], row['Reasons']), axis=1)

In [23]:
processed_df2

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons,Self Verification Result,Cross Verification Result
0,177000,please kick japan out of my country they are s...,na,NaN,NaN,correct,correct
1,177001,once i learned about correctional exercises an...,na,NaN,NaN,correct,correct
2,177002,empty train cars &gt;&gt;,subway,positive,empty train cars,correct,correct
3,177003,that emerald parking lot is about 1 mile walk ...,na,NaN,NaN,correct,correct
4,177004,"that is going to take time, i need car by end ...",car,neutral,"expresses need for a car by end of the year, i...",correct,correct
...,...,...,...,...,...,...,...
995,177995,how about getting all trash off the sidewalk? ...,car,negative,presence of trash on sidewalk and street causi...,correct,correct
996,177996,get trash off the sidewalk. trash corals in th...,car,negative,"congestion caused by cars, lack of space on si...",correct,correct
997,177997,people of philadelphia find trae young's hotel...,car,negative,disruptive behavior of honking car horns all n...,correct,correct
998,177998,ring rain coming back for more and do not you ...,car,positive,expresses joy and appreciation for the time sp...,correct,correct


In [24]:
processed_df2.to_csv('Week 11 result_177000_177999(after cross-verification).csv', index=False)

In [25]:
processed_df2['Cross Verification Result'].value_counts().get('correct', 0)

1000